In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.nn.functional as F

In [2]:
df = pd.read_csv("./Iris.csv")
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
df.shape

(150, 6)

In [4]:
le = LabelEncoder()
df['Species'] = le.fit_transform(df['Species'])
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,0
1,2,4.9,3.0,1.4,0.2,0
2,3,4.7,3.2,1.3,0.2,0
3,4,4.6,3.1,1.5,0.2,0
4,5,5.0,3.6,1.4,0.2,0


In [5]:
cuda_availability = torch.cuda.is_available()
device = torch.device('cuda' if cuda_availability else 'cpu')

In [6]:
class iris(Dataset):
    def __init__(self, x, y) -> None:
        super().__init__()
        self.x = torch.tensor(x.values, dtype=torch.float32, device=device)
        self.y = torch.tensor(y.values.flatten(), dtype=torch.long, device=device)

    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        data = self.x[index]    
        label = self.y[index]
        return data, label

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df[['Species']], test_size=0.2, random_state=32)
train_data = iris(X_train, y_train)
test_data = iris(X_test, y_test)

In [8]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=4, shuffle=False)

In [9]:
class iris_model(nn.Module):
    def __init__(self, in_features, h1, out_features) -> None:
        super().__init__()
        self.fc1 = nn.Linear(in_features,h1)
        self.fc2 = nn.Linear(h1,out_features)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [10]:
model = iris_model(in_features=X_train.shape[1], h1=32, out_features=3)
model = model.to(device)
epoch = 100
criterion = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(), lr=1e-2)

for i in range(epoch):
    losses = 0
    for idx, data in enumerate(train_loader):
        epoch_loss = 0
        x, y = data
        optimiser.zero_grad()
        res = model(x)
        #print(res, y)
        loss = criterion(res, y)
        loss.backward()
        optimiser.step()
        losses += loss.item()
        epoch_loss += loss.item()

    #print(f'At epoch {i}, cumm. loss is {losses}')
    #print(f'At epoch {i}, current loss is {loss.item()}')
    print(f'At epoch {i}, current epoch loss is {epoch_loss}')

At epoch 0, current epoch loss is 1.2582753896713257
At epoch 1, current epoch loss is 1.490930438041687
At epoch 2, current epoch loss is 2.070891857147217
At epoch 3, current epoch loss is 1.290015697479248
At epoch 4, current epoch loss is 0.507476270198822
At epoch 5, current epoch loss is 0.5929914712905884
At epoch 6, current epoch loss is 0.7882558703422546
At epoch 7, current epoch loss is 0.42484143376350403
At epoch 8, current epoch loss is 0.5463254451751709
At epoch 9, current epoch loss is 0.4394073188304901
At epoch 10, current epoch loss is 0.4447683095932007
At epoch 11, current epoch loss is 0.34763529896736145
At epoch 12, current epoch loss is 0.5504084229469299
At epoch 13, current epoch loss is 0.3152860105037689
At epoch 14, current epoch loss is 0.2901690900325775
At epoch 15, current epoch loss is 0.36539116501808167
At epoch 16, current epoch loss is 0.3076247274875641
At epoch 17, current epoch loss is 0.21601657569408417
At epoch 18, current epoch loss is 0.2

In [11]:
from sklearn.metrics import accuracy_score

with torch.no_grad():
    res = []
    for idx, data in enumerate(test_loader):
        x, y = data
        output = model(x)
        _, pred = torch.max(output.data, 1)
        res = np.append(res, pred.cpu().detach().numpy())

accuracy_score(y_test, res)

1.0